In [10]:
%matplotlib inline
import math
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import matplotlib.pyplot as plt
import missingno as msno
import random

df = pd.read_csv('Test_HR_Employee_Attrition.csv')
#enable full view of pandas dataframes
pd.set_option('display.max_columns', None)

In [11]:
## DATA QUALITY ##

df.Gender = df.groupby(['JobRole','MaritalStatus'])['Gender'].apply(lambda x: x.fillna(x.mode().iloc[0]))

df.YearsAtCompany = df.groupby(['JobRole','Gender','MaritalStatus'])['YearsAtCompany'].apply(lambda x: x.fillna(x.mean()))

bt_dic = df.BusinessTravel.value_counts(normalize=True).to_dict()
def give_random():
    return random.choices(list(bt_dic),list(bt_dic.values()))[0]
df.loc[df.BusinessTravel.isnull(),'BusinessTravel'] = df.loc[df.BusinessTravel.isnull(),'BusinessTravel'].apply(lambda x: give_random())

#df.loc[:,'PerformanceRating'] = df.groupby(['Education','JobLevel'])['PerformanceRating'].apply(lambda x: x.fillna(x.mode().iloc[0]))
df.loc[:,'PerformanceRating'] = df.PerformanceRating.fillna((df.PerformanceRating).mode())

df.loc[:,'Age']=df.groupby(['Gender','JobSatisfaction','EnvironmentSatisfaction','StockOptionLevel'])['Age'].apply(lambda x: x.fillna(x.mean()))
df.loc[:,'Age_cat'] = pd.cut(df.Age,
                                bins=[17,25,35,45,55,60],
                                labels=['Very low','low','Medium','High','Very High'])
df.loc[:,'MonthlyIncome'] = df.groupby(['Gender', 'Age_cat'])['MonthlyIncome'].apply(lambda x: x.fillna(x.mean()))

df.drop(columns = 'TrainingTimesLastYear', inplace = True)

#Elimino due colonne irrilevanti #addByAgne
df.drop(columns=['StandardHours','Over18', 'Age_cat'], inplace = True)

#Elimino colonne inconsistent
df.drop(columns = ['YearsInCurrentRole','YearsSinceLastPromotion','YearsWithCurrManager','TotalWorkingYears'], inplace=True)
df.drop(columns = ['DailyRate', 'HourlyRate','MonthlyRate'],inplace=True)
#Elimino riga vuota residua
#df.dropna(inplace=True)
df.loc[:,'Age'].fillna(np.mean(df.Age), inplace = True)
df.loc[:,'MonthlyIncome'].fillna(np.mean(df.MonthlyIncome), inplace = True)

In [12]:
## TRANSFORMATON ##

#Age in integers
df.loc[:,'Age'] = df.Age.round(1)
#Trasformazione in binari di tre attributi categorici 
df['Attrition'] = (df['Attrition'] == "Yes").astype(int)
df['OverTime'] = (df['OverTime'] == "Yes").astype(int)
df['Gender'] = (df['Gender'] == "Male").astype(int)

#Discretizzazione della variabile DistanceFromHome
for i, row in df.iterrows():
    if row['DistanceFromHome'] <= 8: #Proximate
        df.loc[i, 'DistanceFromHome'] = 1 
    if row['DistanceFromHome'] > 8 and row['DistanceFromHome'] <= 15: #Near
        df.loc[i, 'DistanceFromHome'] = 2
    if row['DistanceFromHome'] > 15 and row['DistanceFromHome'] <= 22: #Intermediate
        df.loc[i, 'DistanceFromHome'] = 3
    if row['DistanceFromHome'] > 22 and row['DistanceFromHome'] <= 29: #Far
        df.loc[i, 'DistanceFromHome'] = 4



In [13]:
## CORRELATION ##

for i, row in df.iterrows():
    if row['JobRole'] == 'Manager':
        if row['Department'] == 'Sales':
            df.loc[i, 'JobRole'] = 'Manager Sales'
        if row['Department'] == 'Human Resources':
            df.loc[i, 'JobRole'] =  'Manager HR'
        if row['Department'] == 'Research & Development':
            df.loc[i, 'JobRole'] =  'Manager R&D'
            
df.drop(columns = 'Department', inplace=True)

In [1]:
df.to_csv('Datasource_TEST.csv', index = False) #sistema di numerazione in previsione delle modifiche future, per essere tutti al pari

NameError: name 'df' is not defined